# Predict Food Delivery Time

### 문제설명
이번 과제를 통해 여러분은 음식배달 서비스(배민, 쿠팡이츠 등)를 위한 예측모델을 만들게 될 것입니다! 이 모델이 예측하는 값은 “음식배달에 걸리는 시간"입니다. 배달시간을 정확하게 예측하는 것은 사용자의 경험에 많은 영향을 미치게 됩니다.\
예측된 배달시간보다 실제 배달시간이 더 걸린 경우(under-prediction)가 반대의 경우(over-prediction)보다 두 배로 사용자의 경험에 안 좋은 영향을 준다고 알려져 있습니다.\
가능한 실제 배달시간과 가까운 값을 예측하되 동시에 under-prediction을 최소화하는 것이 좋은 예측모델입니다.

### 학습/테스트 데이터
파일 “predict_delivery_time.csv”는 다음과 같은 속성들을 가지고 있습니다.

- Restaurant, Location, Cuisines, AverageCost, MinimumOrder, Rating, Votes, Reviews 속성들을 모델의 입력속성으로 사용하세요. 모델의 학습목표는 DeliveryTime입니다.
- 이 데이터에서 랜덤하게 20%를 추출해서 테스트 데이터로 사용하고 나머지는 학습데이터로 사용하세요.

### 제출할 결과물
- 간단한 요약문
- 데이터 전처리와 속성 생성에 대한 간단한 설명
- 학습을 위해서 어떤 모델을 사용했는지 그리고 어떠한 손실함수를 사용했는지를 간단히 설명
- 테스트 데이터에 대한 평가지표들 (아래 두가지를 반드시 포함할 것)
- Mean Absolute Error (MAE)
- Under-prediction의 비율 (under-prediction 개수 / 테스트 데이터의 샘플수)
- 모델 학습에 사용한 Jupyter notebook 파일

## 1. 큰 그림을 봅니다 (look at the big picture).

이번 과제를 통해 여러분은 음식배달 서비스(배민, 쿠팡이츠 등)를 위한 예측모델을 만들게 될 것입니다! \
이 모델이 예측하는 값은 “음식배달에 걸리는 시간"입니다. 배달시간을 정확하게 예측하는 것은 사용자의 경험에 많은 영향을 미치게 됩니다.

예측된 배달시간보다 실제 배달시간이 더 걸린 경우(under-prediction)가 반대의 경우(over-prediction)보다 두 배로 사용자의 경험에 안 좋은 영향을 준다고 알려져 있습니다.\
가능한 실제 배달시간과 가까운 값을 예측하되 동시에 under-prediction을 최소화하는 것이 좋은 예측모델입니다.

### 풀어야할 문제 : 음식배달 데이터를 활용해 음식 배달 시간 예측 모델을 만드는 것

실제 배달시간과 가까운 값을 예측: 지도학습, 회귀모델
동시에 under-prediction을 최소화하는 것이 좋은 예측모델 -> 이건 어떻게 풀 수 있을까?

### 성능측정 지표
Mean Absolute Error (MAE)
Under-prediction의 비율 (under-prediction 개수 / 테스트 데이터의 샘플수)

## 2. 데이터를 구합니다 (get the data).

In [1]:
# import common libraries
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import seaborn as sns


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [7]:
food = pd.read_csv('predict_delivery_time.csv')

### 데이터 톺아보기

In [8]:
food.head()

,Restaurant,Location,Cuisines,AverageCost,MinimumOrder,Rating,Votes,Reviews,DeliveryTime
0,ID6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12.0,4.0,30
1,ID2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,50,3.5,11.0,4.0,30
2,ID1595,Mumbai Central,"Italian, Street Food, Fast Food",150,50,3.6,99.0,30.0,65
3,ID5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,99,3.7,176.0,95.0,30
4,ID6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,99,3.2,521.0,235.0,65


### 여기서 다루는 변수는 다음과 같다.

- Restaurant: A unique ID that represents a restaurant.
- Location: The location of the restaurant.
- Cuisines: The cuisines offered by the restaurant.
- Average_Cost: The average cost for one person/order.
- Minimum_Order: The minimum order amount.
- Rating: Customer rating for the restaurant.
- Votes: The total number of customer votes for the restaurant.
- Reviews: The number of customer reviews for the restaurant.
- Delivery_Time: The order delivery time of the restaurant. (Target Classes)

In [9]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Restaurant    11094 non-null  object 
 1   Location      11094 non-null  object 
 2   Cuisines      11094 non-null  object 
 3   AverageCost   11094 non-null  object 
 4   MinimumOrder  11094 non-null  int64  
 5   Rating        9903 non-null   object 
 6   Votes         9020 non-null   float64
 7   Reviews       8782 non-null   float64
 8   DeliveryTime  11094 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 780.2+ KB


In [10]:
food.describe()

,MinimumOrder,Votes,Reviews,DeliveryTime
count,11094.000000,9020.000000,8782.000000,11094.000000
mean,53.344511,244.544457,123.247893,37.056066
std,18.551245,555.094733,321.025216,12.447496
min,0.000000,3.000000,1.000000,10.000000
25%,50.000000,19.000000,7.000000,30.000000
50%,50.000000,63.000000,26.000000,30.000000
75%,50.000000,216.000000,91.000000,45.000000
max,500.000000,9054.000000,6504.000000,120.000000


In [13]:
food.describe(include='O')

,Restaurant,Location,Cuisines,AverageCost,Rating
count,11094,11094,11094,11094,9903
unique,7480,35,2179,26,32
top,ID7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,200,3.7
freq,22,947,850,3241,869


- 간략하게 훑어보고 알 수 있는 사실
1. target label: DeliveryTime 연속변수이며, 최소 10에서 120까지 분포되어 있어 분 단위일 것으로 예상됩니다.
2. AverageCost, Rating 은 numerical 같이 생겼는데 object라고 뜹니다. 어떤 value들이 들어가 있는지 잘 봐야할 것 같습니다.
3. Rating, Votes, Reviews에는 결측치가 상당수 있습니다.
4. Reviews의 분포와 feature에 대한 설명을 보니 점수가 아니라 리뷰의 갯수인 것 같습니다. negative/positive 내용이 구분되어있지 않습니다.
5. Votes 역시 feature의 설명을 보니 긍정적인 의미의 vote의 갯수인지, 그냥 투표를 한 갯수를 의미하는 건지 잘 모르겠네요.
6. object인 변수도 간단하게 살펴보았는데, unique 값이 26개인 feature 부터 7480개인 feature까지 다양합니다. AverageCost와 Rating은 빈출값만 봐도 numerical 한 데이터 같은데 왜 object인지 살펴볼 필요가 있습니다.

## 3. 데이터로부터 통찰을 얻기 위해 탐색하고 시각화합니다 (discover and visualize the data to gain insights).

### 3-1. 데이터 탐색

### 3-2. 데이터 cleaning

### 3-3. 데이터 시각화

### 3-4. 결측치 처리하기

### 3-5. 데이터 스케일링

### 3-6. 상관관계 살펴보기 

## 4. 머신러닝 알고리즘을 위해 데이터를 준비합니다 (prepare the data for Machine Learning algorithms).

### 4-1. 변수 만들기

### 4-2. 트레인-테스트 분리

## 5. 모델을 선택하고 훈련시킵니다 (select a model and train it).

### 5-1. linear regression

### 5-2. Decision Tree

### 5-3. Random Forest 

## 6. 모델을 상세하게 조정합니다 (fine-tune your model).

### 6-1. Grid Search 

### 6-2. Randomized Search

## 7. 솔루션을 제시합니다 (present your solution).

### 특성 중요도, 에러 분석

### 최종 평가

## 8. 시스템을 론칭하고 모니터링하고 유지 보수합니다 (launch, monitor, and maintain your system).